In [1]:
using Distributions
using Plots
using PGFPlotsX
using LaTeXStrings
using LinearAlgebra
using Colors
using JLD
using Parameters
using Random
pgfplotsx()
push!(PGFPlotsX.CUSTOM_PREAMBLE, raw"\usepgfplotslibrary{fillbetween}");

In [2]:
T = 80
n_steps = 50
gridman = LinRange(0, 1, n_steps);

### Heatmap

In [135]:
experiment = JLD.load("jlds/experiment.jld")


Dict{String, Any} with 6 entries:
  "efe_grids"  => Any[19.9776, 20.0538, 20.1301, 20.2064, 20.2823, 20.3576, 20.…
  "points"     => [0.528234 0.0 … 0.795918 0.77551; 0.543932 0.0 … 0.22449 0.22…
  "inst_grids" => Any[19.7641, 19.8398, 19.9157, 19.9915, 20.067, 20.1418, 20.2…
  "responses"  => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.0, 1.…
  "idxs"       => Any[CartesianIndex(1, 1), CartesianIndex(50, 50), CartesianIn…
  "epi_grids"  => Any[0.213498, 0.213928, 0.214377, 0.214845, 0.215328, 0.21582…

In [136]:
efe_grids = experiment["efe_grids"]
efe_grids = reshape(efe_grids, n_steps,n_steps, T);
efe_grids[efe_grids .== Inf] .= maximum(efe_grids[efe_grids .!= Inf])

epi_grids = experiment["epi_grids"]
epi_grids = reshape(epi_grids, n_steps,n_steps, T);

inst_grids = experiment["inst_grids"]
inst_grids = reshape(inst_grids, n_steps,n_steps, T);

idxs = experiment["idxs"]
points = experiment["points"];

In [137]:
index = 80
x = y = gridman

target = [[0.8], [0.2]]
current = map(x -> [first(x)], points[:,index])


axis1 = @pgf Axis(
    {
        title="EFE landscape at time $(index)",
        xlabel=L"g_s",
        ylabel=L"g_n",
        zlabel="hz",
        legend_pos = "north east",
        legend_cell_align="{left}",
        grid = "major",
        view = (0, 90),
        colorbar,
        "colormap/hot",
        colorbar_style={"xlabel=EFE, xticklabel pos=upper"}
    },
    Plot({only_marks, color="green", mark_size=3.0, mark="*"}, 
    Coordinates(target...)), LegendEntry("Target"),
    Plot({only_marks, color="black", mark_size=3.0, mark="*"}, 
    Coordinates(current...)), LegendEntry("Current"),
    Plot3(
        {
            surf,
            shader = "flat",
        },
        Coordinates(x, y, efe_grids[:, :, index]))
    )
pgfsave("tmp/heatmap_$(index).pdf", axis1)
pgfsave("tikz/heatmap_$(index).tikz", axis1)

### EFE plots

In [138]:
# Get the values of queried points for all time steps
epi_mins = [epi_grids[:,:,i][idxs[i]] for i in 1:T]
inst_mins = [inst_grids[:,:,i][idxs[i]] for i in 1:T]

# Get rid of infs for inhibition of return
epi_means = [mean(epi_grids[:,:,i][isfinite.(epi_grids[:,:,i])]) for i in 1:T]
inst_means = [mean(inst_grids[:,:,i][isfinite.(inst_grids[:,:,i])]) for i in 1:T];

In [139]:
plt_agent = @pgf GroupPlot(
    # group plot options
    {
        width="15cm", height="8cm",

        group_style = {
            group_size = "2 by 1",
            horizontal_sep = "1.5cm",
        },
    },
    # first
    {xlabel=L"t",
           ylabel="value",
           legend_pos = "north east",
           legend_cell_align="{left}",
           grid = "major",
        #    title="TODO: TITLE",
           legend_style = "{nodes={scale=0.8, transform shape}}",
    },
    Plot(
        {no_marks,color="blue"}, Coordinates(collect(1:T), epi_mins)
    ), LegendEntry("Epistemic value"),
    # second
    {xlabel=L"t",
           legend_pos = "south east",
           legend_cell_align="{left}",
           grid = "major",
        #    title="Dataset $(index)",
           legend_style = "{nodes={scale=0.8, transform shape}}",
    },
    Plot(
        {no_marks,color="blue"}, Coordinates(collect(1:T), inst_mins)
    ), LegendEntry("Instrumental value"),
)

pgfsave("tmp/grouped_agent.pdf", plt_agent)
pgfsave("tikz/grouped_agent.tikz", plt_agent)

In [140]:
# Plot of user pref function
include("../../src/environment/user.jl")
n_ticks = 100 # How finely to grid the preference function

x = y = LinRange(0,1,n_ticks)

z = zeros(n_ticks,n_ticks);
for i in 1:n_ticks
    for j in 1:100
	z[i,j] = generate_user_response([i/n_ticks,j/n_ticks],binary=false)
    end
end;

pref_map= @pgf Axis(
    {
        title="User preference function",
        xlabel=L"g_s",
        ylabel=L"g_n",
        zlabel="hz",
        legend_pos = "north east",
        legend_cell_align="{left}",
        grid = "major",
        view = (0, 90),
        colorbar,
        "colormap/hot",
        colorbar_style={"xlabel=θ, xticklabel pos=upper"}
    },
    Plot3(
        {
            surf,
            shader = "flat",
        },
        Coordinates(x, y, z))
    )
pgfsave("tmp/pref_map.pdf", pref_map)
pgfsave("tikz/pref_map.tikz", pref_map)

### Batch Plots

In [142]:
using Colors
experiment_1 = JLD.load("jlds/batch_experiment_1.jld")
experiment_2 = JLD.load("jlds/batch_experiment_2.jld")

# Heatmap of positive user appraisals
responses_1 = experiment_1["responses"]
responses_2 = experiment_2["responses"]
resp = vcat(responses_1,responses_2);

In [143]:
y = 1:T 
x = 1:T+1

resp_map= @pgf Axis(
    {
        title="Positive Appraisals",
        xlabel="Time Index",
        ylabel="Agent number",
        grid = "minor",
        view = (0, 90), 
        colormap = "{blackwhite}{color=(black) color=(yellow)}"
    },
    Plot3(
        {
            surf,
            shader = "flat corner",
        },
        Coordinates(x, y, resp')),
    )
pgfsave("tmp/resp_map.pdf", resp_map)
pgfsave("tikz/resp_map.tikz", resp_map)

In [133]:
using StatsBase: Histogram, fit, normalize
h = fit(Histogram, Float64.(idxs), nbins=100, closed = :left)

shift = 2
plt_histogram = @pgf Axis(
    {
        xlabel="Time Index",
        ylabel="Number of Postive responses",
        title="Ensemble Performance",
        yticklabel_style={
        "/pgf/number format/fixed,
        /pgf/number format/precision=0"
        },
        grid="major",
        yminorgrids=true,
        tick_align="outside",
        xtick=range(0, 80; length=5),
        ytick=range(1, 18; length=9),
        xmin =0.0,
        xmax=maximum(idxs)+shift,
        scaled_y_ticks = false,
    },
    Plot({"ybar interval", fill="blue!85", "forget plot"}, Table(h)))
pgfsave("tikz/ensemble_perf.tikz", plt_histogram)
pgfsave("tmp/ensemble_perf.pdf", plt_histogram)

Axis(PGFPlotsX.Options(OrderedCollections.OrderedDict{Any, Any}("xlabel" => "Time Index", "ylabel" => "Number of Postive responses", "title" => "Ensemble Performance", "yticklabel_style" => PGFPlotsX.Options(OrderedCollections.OrderedDict{Any, Any}("/pgf/number format/fixed,\n        /pgf/number format/precision=0" => nothing), true), "grid" => "major", "yminorgrids" => true, "tick_align" => "outside", "xtick" => 0.0:20.0:80.0, "ytick" => 1.0:2.125:18.0, "xmin" => 0.0…), true), Any[Plot(false, false, PGFPlotsX.Options(OrderedCollections.OrderedDict{Any, Any}("ybar interval" => nothing, "fill" => "blue!85", "forget plot" => nothing), true), Table(PGFPlotsX.Options(OrderedCollections.OrderedDict{Any, Any}(), false), TableData([1.0 2.0; 2.0 0.0; … ; 82.0 16.0; 83.0 0.0], nothing, Int64[], true)), Any[])])